# Data Preparation and Cleaning
---

In [106]:
# Basic Libraries
import pandas as pd
import numpy as np

Dataset from Kaggle: **Spotify Dataset 1921-2020, 600k+ Tracks**  
Source: https://www.kaggle.com/datasets/yamaerenay/spotify-dataset-19212020-600k-tracks

As the orginal dataset size was above 100mb (above limit to push on github and work with teamates), the dataset was evenly reduced by 20%. As our end goal is to work with popularity, the goal is to maintain the same proportion of records from each category in the sample as in the original data. In this case, the categories are the popularity ranges, and the code is ensuring that each range is represented proportionally in the reduced dataset.

## Attributes of Audio
---
**id**: id of track  

**name**: name of track

**popularity**: popularity of track in range 0 to 100  

**duration_ms**: duration of songs in ms

**explicit**: whether it contains explicit content or not

**artists**: artists who created the track

**id_artists**: id of artists who created the track  

**release_date**: date of release  

**danceability**: how danceable a song is in range 0 to 1  

**energy**: how energized a song is in range 0 to 1  

**key**: major note of track [0: C, 1: C#/Db, 2: D, …]  

**loudness**: How loud a song is in db  

**mode**: The modality of track, 0 if minor and 1 if major  

**speechiness**: The presence of spoken words in track in range 0 to 1

**acousticness**: How acoustic a track is in range 0 to 1

**instrumentalness**: The absence of vocal sounds in track in range 0 to 1

**liveness**: The presence of audience in track in range 0 to 1  

**valence**: The positiveness of the track in range 0 to 1  

**tempo**: The overall tempo of track in BPM  

**time_signature**: The time signature (4 in almost every track)

In [ ]:
# Load the dataset into a pandas dataframe
df = pd.read_csv('tracks.csv')

# Calculate the histogram of the "popularity" column
hist, bins = np.histogram(df['popularity'], bins=5)

# Calculate the number of records to keep from each bin
n_records_to_keep = (hist * 0.8).astype(int)

# Select the required number of records randomly from each bin
dfs = []
for i in range(len(n_records_to_keep)):
    bin_df = df[(df['popularity'] >= bins[i]) & (df['popularity'] < bins[i+1])]
    dfs.append(bin_df.sample(n=n_records_to_keep[i], random_state=42))

# Concatenate the selected records from all bins to create a new DataFrame
df_new = pd.concat(dfs)

# Write the new DataFrame to a new CSV file
df_new.to_csv('tracks_reduced.csv', index=False)

In [107]:
musicData = pd.read_csv('tracks_reduced.csv')
musicData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 469336 entries, 0 to 469335
Data columns (total 20 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                469336 non-null  object 
 1   name              469281 non-null  object 
 2   popularity        469336 non-null  int64  
 3   duration_ms       469336 non-null  int64  
 4   explicit          469336 non-null  int64  
 5   artists           469336 non-null  object 
 6   id_artists        469336 non-null  object 
 7   release_date      469336 non-null  object 
 8   danceability      469336 non-null  float64
 9   energy            469336 non-null  float64
 10  key               469336 non-null  int64  
 11  loudness          469336 non-null  float64
 12  mode              469336 non-null  int64  
 13  speechiness       469336 non-null  float64
 14  acousticness      469336 non-null  float64
 15  instrumentalness  469336 non-null  float64
 16  liveness          46

In [108]:
# Removing duplicates if they exist
musicData = musicData.drop_duplicates()

In [109]:
# Identifying missing data

missing_data = musicData.isnull().sum()
print("Number of missing values per column:\n", missing_data)

Number of missing values per column:
 id                   0
name                55
popularity           0
duration_ms          0
explicit             0
artists              0
id_artists           0
release_date         0
danceability         0
energy               0
key                  0
loudness             0
mode                 0
speechiness          0
acousticness         0
instrumentalness     0
liveness             0
valence              0
tempo                0
time_signature       0
dtype: int64


---
We notice that there are some entries that do not have a name. To ensure data consistency, we will remove these entries.

In [110]:
# Removing rows with null values
musicData = musicData.dropna()
musicData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 469281 entries, 0 to 469335
Data columns (total 20 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                469281 non-null  object 
 1   name              469281 non-null  object 
 2   popularity        469281 non-null  int64  
 3   duration_ms       469281 non-null  int64  
 4   explicit          469281 non-null  int64  
 5   artists           469281 non-null  object 
 6   id_artists        469281 non-null  object 
 7   release_date      469281 non-null  object 
 8   danceability      469281 non-null  float64
 9   energy            469281 non-null  float64
 10  key               469281 non-null  int64  
 11  loudness          469281 non-null  float64
 12  mode              469281 non-null  int64  
 13  speechiness       469281 non-null  float64
 14  acousticness      469281 non-null  float64
 15  instrumentalness  469281 non-null  float64
 16  liveness          46

---
Our analysis revolves around predicting popularity of music using its features. As a result, it might not be meaningful to use id_artists. Rather we could adapt this to find the number of artists on a track and analyze if that has any impact

In [111]:
# function to count number of artists in a track
def count_artists(artist_str):
    return len(artist_str.split(','))

# Apply the count_artists function to the artists column and store the result in a new column called num_artists
musicData['num_artists'] = musicData['artists'].apply(count_artists)

# verifying it has been added
musicData.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,num_artists
0,623Yfxfk1RjLdGHTeADzP3,Nitar och läder,14,243200,0,['Magnus Uggla'],['30j6YCWLSp59jLF7yIYZWq'],1977,0.541,0.869,...,-2.439,1,0.0567,0.012,0.000000,0.0796,0.750,142.934,4,1
1,77G5CnYygZzIrMbueR5RD7,Chekkili Meeda,6,260000,0,"['P. Susheela', 'V. Ramakrishna']","['0aFGod7DM6b3O5l1AmvFwK', '1dvwl7Qo8coy4VxG10...",1974-12-31,0.543,0.325,...,-22.995,0,0.0524,0.978,0.792000,0.2420,0.796,103.406,4,2
2,0BM2RqUGhIgQVQniajaEGc,2007: Sasabe Pre-Wall International Ocotillo a...,0,368301,0,['Glenn Weyant'],['0QbCmYOetRNu4mwmc68RGl'],2016-05-27,0.236,0.497,...,-19.702,0,0.0863,0.994,0.938000,0.1480,0.037,63.582,4,1
3,1JyMUJT3mQd9GCbNvtFf2P,Pod Naším Oknem,13,198627,0,['Mistříňanka'],['2Fy5r7sT3YMkMWW2zea3Qj'],1994-01-01,0.710,0.390,...,-11.862,1,0.0749,0.742,0.000001,0.1220,0.968,133.718,4,1
4,0V9zWokWpffjMclDom454X,รักคนมีเจ้าของ,19,197320,0,['ไอ..น้ำ'],['6kTljLBKMjT7N66D7T1IB4'],2016-11-10,0.754,0.853,...,-4.852,1,0.0311,0.283,0.000000,0.0887,0.710,115.985,4,1


In [114]:
# extracting release year from date and storing it in a new column
def get_year(date_string):
    return int(date_string[:4])

musicData['year'] = musicData['release_date'].apply(get_year)
musicData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 469281 entries, 0 to 469335
Data columns (total 22 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                469281 non-null  object 
 1   name              469281 non-null  object 
 2   popularity        469281 non-null  int64  
 3   duration_ms       469281 non-null  int64  
 4   explicit          469281 non-null  int64  
 5   artists           469281 non-null  object 
 6   id_artists        469281 non-null  object 
 7   release_date      469281 non-null  object 
 8   danceability      469281 non-null  float64
 9   energy            469281 non-null  float64
 10  key               469281 non-null  int64  
 11  loudness          469281 non-null  float64
 12  mode              469281 non-null  int64  
 13  speechiness       469281 non-null  float64
 14  acousticness      469281 non-null  float64
 15  instrumentalness  469281 non-null  float64
 16  liveness          46

---
We can remove irrelevant data which would are not realated to the audio features of a track: id, name, id_artists, artists

In [115]:
musicData.drop(['id', 'name','artists','id_artists'],axis=1,inplace=True)
musicData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 469281 entries, 0 to 469335
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   popularity        469281 non-null  int64  
 1   duration_ms       469281 non-null  int64  
 2   explicit          469281 non-null  int64  
 3   release_date      469281 non-null  object 
 4   danceability      469281 non-null  float64
 5   energy            469281 non-null  float64
 6   key               469281 non-null  int64  
 7   loudness          469281 non-null  float64
 8   mode              469281 non-null  int64  
 9   speechiness       469281 non-null  float64
 10  acousticness      469281 non-null  float64
 11  instrumentalness  469281 non-null  float64
 12  liveness          469281 non-null  float64
 13  valence           469281 non-null  float64
 14  tempo             469281 non-null  float64
 15  time_signature    469281 non-null  int64  
 16  num_artists       46

In [116]:
# checking if data is valid
musicData.describe()

,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,num_artists,year
count,469281.000000,4.692810e+05,469281.000000,469281.000000,469281.000000,469281.000000,469281.000000,469281.000000,469281.000000,469281.000000,469281.000000,469281.000000,469281.000000,469281.000000,469281.000000,469281.000000,469281.000000
mean,27.578334,2.299850e+05,0.044155,0.563647,0.542439,5.224840,-10.200712,0.659208,0.104881,0.449396,0.113413,0.213992,0.552442,118.465459,3.873500,1.295914,1988.594946
std,18.366320,1.270919e+05,0.205439,0.166195,0.251780,3.517928,5.086349,0.473976,0.179904,0.348656,0.266952,0.184386,0.257641,29.784032,0.472858,0.887235,22.813764
min,0.000000,3.344000e+03,0.000000,0.000000,0.000000,0.000000,-60.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1922.000000
25%,13.000000,1.750530e+05,0.000000,0.453000,0.344000,2.000000,-12.887000,0.000000,0.034000,0.096900,0.000000,0.098300,0.346000,95.549000,4.000000,1.000000,1974.000000
50%,27.000000,2.149070e+05,0.000000,0.577000,0.550000,5.000000,-9.233000,1.000000,0.044300,0.422000,0.000024,0.139000,0.564000,117.363000,4.000000,1.000000,1992.000000
75%,41.000000,2.638000e+05,0.000000,0.686000,0.749000,8.000000,-6.482000,1.000000,0.076400,0.784000,0.009460,0.278000,0.769000,136.335000,4.000000,1.000000,2007.000000
max,98.000000,5.621218e+06,1.000000,0.991000,1.000000,11.000000,5.376000,1.000000,0.971000,0.996000,1.000000,1.000000,1.000000,243.507000,5.000000,58.000000,2021.000000


The following attributes have a range between 0 and 1:

* Danceability
* Energy
* Speechiness
* Acousticness
* Instrumentalness 
* Liveness
* Valence

Looking at the shape of data, all of them conform to the requirements  


---

In [103]:
# pushing cleaned data to a csv
musicData.to_csv('tracks_cleaned.csv',index=False)